In [42]:
import bs4
import lxml
import pandas as pd
import urllib
import csv
import re

from urllib import request

import requests
from lxml import html
import time

### Scrapping du site Century21

In [5]:
base_url = "https://www.century21.fr/annonces/location-appartement/v-paris/"
url = "https://www.century21.fr/annonces/location-appartement/v-paris/page-2/"
request_text = request.urlopen(base_url).read()
page = bs4.BeautifulSoup(request_text, "lxml")

In [6]:
# Fonction pour remplacer la première virgule par un point avant "m2"
def replace_comma_with_dot_before_m2(text):
    return re.sub(r'(\d+),(\d+\s*m2)', r'\1.\2', text, 1)

def delete_m2(text):
    return re.sub(r'(\d+\s*) m2', r'\1', text, 1)

def delete_pieces(text):
    return re.sub(r' (\d+\s*) pièce.*', r'\1', text, 1)

def delete_euro(text):
    return re.sub(r'€', '', text, 1)


def get_info(annonces):
    infos = []
    for annonce in annonces:
        info = annonce.find(class_="c-text-theme-heading-4 tw-text-c21-grey-darker tw-font-semibold")
        prix = annonce.find(class_="c-text-theme-heading-1 is-constant-size-on-mobile tw-mt-2 tablet-landscape:tw-mt-0 tw-whitespace-nowrap")
        line = info.get_text().strip().split("\n")
        res = []
        res.append(line[0].strip())
        res.append(line[2].strip())
        modified_string = line[5].strip()
        modified_string = replace_comma_with_dot_before_m2(modified_string).split(",")
        modified_string[0] = delete_m2(modified_string[0])
        modified_string[1] = delete_pieces(modified_string[1])
        res.append(modified_string[0])
        res.append(modified_string[1])
        modified_string = prix.get_text().strip().split("\n")[0]
        modified_string = modified_string.replace(" ", "")
        modified_string = delete_euro(modified_string)
        res.append(modified_string)
        infos.append(res)
    return infos


annonces = page.find_all(class_="js-the-list-of-properties-list-property")
data = get_info(annonces)
while True:
    a_tags = page.find_all(class_="c-the-pagination-bar__item tw-flex tw-items-center tw-justify-center tw-w-10 tw-h-10 tw-bg-c21-gold tw-text-white tw-transition-colors tw-duration-200")
    print(len(a_tags))
    if len(a_tags) == 1 and a_tags[0].get('aria-label') == "prev":
        break

    for a_tag in a_tags:
        if a_tag.get('aria-label') == "next":
            url = "https://www.century21.fr" + a_tag.get('href')
            request_text = request.urlopen(url).read()
            page = bs4.BeautifulSoup(request_text, "lxml")

    annonces = page.find_all(class_="js-the-list-of-properties-list-property")
    data += get_info(annonces)

1
2
2
1


In [ ]:
filename = 'century21Paris.csv'
labels = ['Ville', 'Arrondissement', 'Surface (m2)', 'Pièce(s)', 'Prix (€)']
data.insert(0, labels)
# Création et écriture dans le fichier CSV
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Écriture de chaque ligne sélectionnée dans le fichier CSV
    for line in data:
        writer.writerow([line[0], line[1], line[2], line[3], line[4]])

print(f'Fichier "{filename}" créé avec succès.')

Fichier "century21Paris.csv" créé avec succès.


### Scrapping de immo-data (statistiques de ventes)

In [58]:
def get_ventes_liste(request_text, verbose=False, empty_elements=False) :
    
    ventes_liste = []
    page = bs4.BeautifulSoup(request_text, "lxml")
    ventes = page.find_all(class_="border-b-gray-100")

    for v in ventes :
        print("----------------------------------------------") if verbose else None

        # Features principaux
        features_list1 = v.find(class_="gap-8")
        print(features_list1.find("p").text) if verbose else None
        print([feature.text for feature in features_list1.find_all("span")]) if verbose else None

        # Features dans les 'boîtes'
        features_list2 = v.find(class_="gap-5")
        features_list3 = []
        for child in features_list2.children :
            print("--------") if verbose else None
            for grandchild in child.children :
                print(grandchild.text) if verbose else None
                features_list3.append(grandchild.text)

        # Tout récupérer sous forme de liste
        print("--------") if verbose else None
        try :
            vente_element = [
                features_list1.find("p").text,                              # Adresse
                features_list1.find("span").text,                           # Type (Appartement, Maison)
                re.sub(r'\D', '', features_list1.find_all("span")[1].text), # Prix
                re.sub(r'\D', '', features_list1.find_all("span")[2].text), # Prix par mois
                features_list3[1],                                          # Nombre de pièces
                re.sub(r'\D', '', features_list3[3]),                       # Surface en m²
                features_list3[5],                                          # Date de vente
            ]
        
            print("--> ", vente_element) if verbose else None
            ventes_liste.append(vente_element)

        except :
            ventes_liste.append([]) if empty_elements else None # On n'a pas toutes les infos

    return ventes_liste

    # Adresse, type, prix, prix_mois, nb_pieces, surface, vendu_le

In [59]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

base_url = "https://www.immo-data.fr/explorateur/transaction/recherche?minprice=0&maxprice=25000000&minpricesquaremeter=0&maxpricesquaremeter=40000&propertytypes=0%2C1%2C2%2C4%2C5&minmonthyear=Janvier%202014&maxmonthyear=Juin%202023&nbrooms=1%2C2%2C3%2C4%2C5&minsurface=0&maxsurface=400&minsurfaceland=0&maxsurfaceland=50000&center=2.35191%3B48.866216&zoom=17.40427344023878"

driver = webdriver.Chrome(ChromeDriverManager().install())
# driver = webdriver.Chrome() 
driver.get(base_url)
time.sleep(10)
page_html = driver.page_source
page = bs4.BeautifulSoup(page_html, 'lxml')

# ventes = page.find_all("div", class_="border-b border-b-gray-100")

for vente in get_ventes_liste(page_html, verbose=False, empty_elements=False) :
    print(vente)
driver.quit()

C:\Users\sunse\AppData\Local\Temp\ipykernel_65172\595069821.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


['76 RUE RÉAUMUR - PARIS', 'Appartement', '547200', '10944', '3', '50', '19/06/2023']
['11 RUE DU PONCEAU - PARIS', 'Appartement', '98170', '10908', '1', '9', '02/06/2023']
['3 RUE DU PONCEAU - PARIS', 'Appartement', '165000', '8250', '1', '20', '19/04/2023']
['9 RUE DU PONCEAU - PARIS', 'Appartement', '478350', '11959', '2', '40', '13/04/2023']
['76 RUE RÉAUMUR - PARIS', 'Appartement', '513333', '7897', '3', '65', '23/01/2023']
['19 RUE DE PALESTRO - PARIS', 'Appartement', '100000', '3571', '1', '28', '20/01/2023']
['3 RUE DU PONCEAU - PARIS', 'Appartement', '228000', '11400', '1', '20', '06/01/2023']
['25 RUE DE PALESTRO - PARIS', 'Appartement', '840000', '11507', '3', '73', '16/12/2022']
['3 PASSAGE DE LA TRINITÉ - PARIS', 'Appartement', '501500', '9462', '2', '53', '23/11/2022']
['193 RUE SAINT-DENIS - PARIS', 'Appartement', '565000', '12283', '2', '46', '03/10/2022']
['15 PASSAGE DE LA TRINITÉ - PARIS', 'Appartement', '435550', '12810', '2', '34', '05/09/2022']
['25 RUE DE PALESTR